# Discretising infinite integrals

For the multilayer finite dipole method (FDM), we need to calculate the electric potential and field at the surface using integrals of the form
$$
\int_0^\infty\beta(k)f(k)dk
$$
where $f(k)$ is a weighting function
$$
f(k)=\begin{equation}
\left\{ 
  \begin{aligned}
    e^{-2k}, & \text{ for potential},\\
    -k e^{-2k}, & \text{ for electric field}.
  \end{aligned}
  \right.
\end{equation}
$$

`scipy.integrate.quad_vec` can perform infinite integrals, but it relies on adaptive quadrature and it can't be compiled using `numba`.
It would be faster if we could truncate the infinite integral then perform fixed quadrature taking advantage of `numpy`/`numba`'s fast vectorised array routines.

We don't always know the form of $\beta(k)$, but we can assume that when the weighting function $f(x)$ converges then the whole integrand must also converge (unless $\beta(k)>>f(k)$, which seems unlikely).

We can quantify the proportion of the total integral accounted for for a particular truncation as
$$
\frac{\int_0^{k_{max}}f(k)dk}{\int_0^\infty f(k)dk} = \begin{equation}
\left\{ 
  \begin{aligned}
    1 - e^{-2k_{max}}, & \text{ for potential},\\
    1 - (2k_{max}+1)e^{-2k_{max}}, & \text{ for electric field},
  \end{aligned}
  \right.
\end{equation}
$$
where $k_{max}$ is the value of $k$ at which to truncate the integral.
This will yield a good approximation to the accuracy of the full integral.

Once a truncation point has been set, a definite integral can be estimated using a simple quadrature algoritm such as the trapezium rule.
However both forms of $f(x)$ are strongly weighted towards values of $k$ close to zero, which could mean fixed width quadrature could be inefficient, or introduce large errors (in order to capture the sharp features close to $k=0$ with enough detail, a narrow bin width would need to be chosen, but that would mean many bins further from zero capturing the approximately flat tail of the distribution.

To alleviate this problem, the widths of the bins can be varied according to the known properties of the weighting functions $f(x)$, such that each bin contains an equal proportion of the total area of the integral between $0$ and $k_{max}$.
The edges of the bins can be found by solving
$$
\int_0^{k_{p}} f(k)dk = p \int_0^{k_{max}} f(k)dk
$$
where $k_{p}$ is a limit ($<k_{max}$) of a smaller section of the total integral, and $p$ is the proportion of the total integral which it accounts for.

For electric potential, the value has an algebraic form given by
$$
k_p = -\frac{\ln{\left(1-p\left(1-e^{-2k_{max}}\right)\right)}}{2}.
$$

For electric field, the value has no convenient algebraic form and must be found by inverting the equation
$$
p = \frac{1 - (2k_p + 1)e^{-2k_p}}{1 - (2k_{max} + 1)e^{-2k_{max}}}.
$$

The cells below contain interactive widgets which demonstrate the concepts discussed above.

In [1]:
import numpy as np
from scipy.integrate import quad_vec
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [3]:
@widgets.interact(N=(1, 128, 1), x_max=(0.0, 5))
def f_plot(N, x_max):
    def f(x):
        return np.exp(-2*x)

    x_lims = -1, 5
    y_lims = -0.1, 1.1
    
    x_line = np.linspace(*x_lims, 512)
    y_line = f(x_line)
    
    x = -np.log(1 - np.linspace(0, 1, N+1) * (1 - np.exp(-2 * x_max))) / 2
    total, _ = quad_vec(f, 0, x_max)
    components = [quad_vec(f, x[i], x[i+1])[0] for i in range(N)]
    
    fig, ax = plt.subplots()
    
    ax.plot(x_line, y_line)
    ax.scatter(x, f(x))
    
    ax.axvline(0)
    ax.axvline(x_max)
    text = [ax.text(x[i], f(x[i]), f"{N*components[i]/total:g}") for i in range(N)]
    
    ax.set(
        xlim=x_lims,
        ylim=y_lims,
    )
    fig.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=64, description='N', max=128, min=1), FloatSlider(value=2.5, description…

In [5]:
@widgets.interact(N=(1, 128, 1), x_max=(0.0, 5))
def g_plot(N, x_max):
    def g(x):
        return -x * np.exp(-2*x)
    
    x_lims = -1, 5
    y_lims = -0.3, 0.1
    
    x_line = np.linspace(*x_lims, 512)
    y_line = g(x_line)
    
    _y = np.linspace(0, x_max, 64)
    _x = (1 - (2 * _y + 1) * np.exp(-2 * _y)) / (1 - (2 * x_max + 1) * np.exp(-2 * x_max))
    x = np.interp(np.linspace(0, 1, N+1), _x, _y)
    total, _ = quad_vec(g, 0, x_max)
    components = [quad_vec(g, x[i], x[i+1])[0] for i in range(N)]
    
    fig, ax = plt.subplots()
    
    ax.plot(x_line, y_line)
    ax.scatter(x, g(x))
    
    ax.axvline(0)
    ax.axvline(x_max)
    text = [ax.text(x[i], g(x[i]), f"{N*components[i]/total:g}") for i in range(N)]
    
    ax.set(
        xlim=x_lims,
        ylim=y_lims,
    )
    fig.tight_layout()
    plt.show()

interactive(children=(IntSlider(value=64, description='N', max=128, min=1), FloatSlider(value=2.5, description…